In [ ]:
! git clone https://github.com/seshuad/IMagenet
! ls 'IMagenet/tiny-imagenet-200/'

Cloning into 'IMagenet'...
remote: Enumerating objects: 120594, done.
remote: Total 120594 (delta 0), reused 0 (delta 0), pack-reused 120594
Receiving objects: 100% (120594/120594), 212.68 MiB | 22.63 MiB/s, done.
Resolving deltas: 100% (1115/1115), done.
Updating files: 100% (120206/120206), done.
test  train  val  wnids.txt  words.txt


In [ ]:
import time
import imageio
import numpy as np
from keras.utils import to_categorical

path = 'IMagenet/tiny-imagenet-200/'

def get_id_dictionary():
    id_dict = {}
    for i, line in enumerate(open(path + 'wnids.txt', 'r')):
        id_dict[line.strip()] = i
    return id_dict

def get_class_to_id_dict():
    id_dict = get_id_dictionary()
    all_classes = {}
    result = {}
    for i, line in enumerate(open(path + 'words.txt', 'r')):
        n_id, word = line.split('\t')[:2]
        all_classes[n_id] = word.strip()
    for key, value in id_dict.items():
        result[value] = (key, all_classes[key])
    return result

def get_data(id_dict):
    print('starting loading data')
    train_data, test_data = [], []
    train_labels, test_labels = [], []
    t = time.time()
    for key, value in id_dict.items():
        for i in range(500):
            img_path = f'{path}train/{key}/images/{key}_{i}.JPEG'
            img = imageio.imread(img_path)
            if img.shape != (64, 64, 3):
                continue
            train_data.append(img)
            train_labels.append(value)

    for line in open(f'{path}val/val_annotations.txt'):
        img_name, class_id = line.split('\t')[:2]
        img = imageio.imread(f'{path}val/images/{img_name}')
        if img.shape != (64, 64, 3):
            continue
        test_data.append(img)
        test_labels.append(id_dict[class_id])

    print('finished loading data, in {} seconds'.format(time.time() - t))
    return np.array(train_data), to_categorical(train_labels, 200), np.array(test_data), to_categorical(test_labels, 200)

class_to_id = get_class_to_id_dict()

train_data, train_labels, test_data, test_labels = get_data(get_id_dictionary())
print("train data shape: ", train_data.shape)
print("train label shape: ", train_labels.shape)
print("test data shape: ", test_data.shape)
print("test_labels.shape: ", test_labels.shape)


starting loading data


<ipython-input-2-2f3a657d405e>:33: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = imageio.imread(img_path)
<ipython-input-2-2f3a657d405e>:41: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = imageio.imread(f'{path}val/images/{img_name}')


finished loading data, in 43.862701416015625 seconds
train data shape:  (98179, 64, 64, 3)
train label shape:  (98179, 200)
test data shape:  (9832, 64, 64, 3)
test_labels.shape:  (9832, 200)


In [ ]:
def shuffle_data(train_data, train_labels ):
    size = len(train_data)
    train_idx = np.arange(size)
    np.random.shuffle(train_idx)

    return train_data[train_idx], train_labels[train_idx]

train_data, train_labels = shuffle_data(train_data, train_labels)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [ ]:
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers[:-4]:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(200, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.2
)

val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

train_generator = train_datagen.flow(train_data, train_labels, batch_size=32)
val_generator = val_datagen.flow(test_data, test_labels, batch_size=32)


optimizer = Adam(learning_rate=0.0001)


model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True)

In [ ]:
print(model.summary())


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_data) // 32,
    epochs=30,
    validation_data=val_generator,
    validation_steps=len(test_data) // 32,
    callbacks=[early_stop, checkpoint]
)


Epoch 1/30
3068/3068 [==============================] - 121s 39ms/step - loss: 1.5850 - accuracy: 0.6082 - val_loss: 2.4957 - val_accuracy: 0.5024
Epoch 2/30
3068/3068 [==============================] - 121s 39ms/step - loss: 1.5703 - accuracy: 0.6117 - val_loss: 2.5216 - val_accuracy: 0.5029
Epoch 3/30
3068/3068 [==============================] - 125s 41ms/step - loss: 1.5472 - accuracy: 0.6176 - val_loss: 2.4921 - val_accuracy: 0.5060
Epoch 4/30
3068/3068 [==============================] - 124s 40ms/step - loss: 1.5333 - accuracy: 0.6202 - val_loss: 2.4893 - val_accuracy: 0.5077
Epoch 5/30
3068/3068 [==============================] - 123s 40ms/step - loss: 1.5114 - accuracy: 0.6266 - val_loss: 2.5727 - val_accuracy: 0.5062
Epoch 6/30
3068/3068 [==============================] - 126s 41ms/step - loss: 1.5049 - accuracy: 0.6273 - val_loss: 2.4994 - val_accuracy: 0.5130
Epoch 7/30
3068/3068 [==============================] - 126s 41ms/step - loss: 1.4908 - accuracy: 0.6314 - val_loss: 2